In [1]:
!pip install azure-ai-textanalytics==5.3.0

  Using cached azure_core-1.32.0-py3-none-any.whl.metadata (39 kB)
  Using cached azure_common-1.1.28-py2.py3-none-any.whl.metadata (5.0 kB)
  Using cached isodate-0.7.2-py3-none-any.whl.metadata (11 kB)
Using cached azure_common-1.1.28-py2.py3-none-any.whl (14 kB)
Using cached azure_core-1.32.0-py3-none-any.whl (198 kB)
Using cached isodate-0.7.2-py3-none-any.whl (22 kB)


In [2]:
import requests, uuid, json,os
from dotenv import load_dotenv
from enum import Enum
from openai import AzureOpenAI  
from datetime import datetime


from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient #azure-ai-textanalytics==5.3.0 


In [5]:
class AIServiceType(Enum):
    OPENAI_LANGUAGEAPI = 'Language'
    OPENAI_DALLE = 'Dall-e'
    OPENAI_WHISPER = 'Whisper'
    SPEECH_TO_TEXT='SpeechToText'
    TEXT_TO_SPEECH='TextToSpeech'
    CUSTOM_VOICE='CustomVoice'
    CONTENT_SAFETY='ContentSafety'
    COMPUTER_VISION='ComputerVision'
    LANGUAGE='Language'
    TEXT_TRANSLATION='TextTranslation'
    DOCUMENT_TRANSLATION='DocumentTranslation'

In [16]:
class AzureAIExamples:
    def __init__(self, aiservice_type):
        _ = load_dotenv()
        self.key = os.environ['AZUREAI_TOKEN']
        self.location = os.environ['AZURE_LOCATION']
        self.aiservice_type = aiservice_type
        self.based_endpoint = os.environ['BASED_ENDPOINT']
        self.cognitive_service_url = os.environ['AZUREAI_COGNITIVESERVICES_URL']

        self.params = {
            'api-version': '3.0'
        }

        self.headers = {
            'Ocp-Apim-Subscription-Key': self.key,
            # location required if you're using a multi-service or regional (not global) resource.
            'Ocp-Apim-Subscription-Region': self.location,
            'Content-type': 'application/json',
            'X-ClientTraceId': str(uuid.uuid4())
        }

    
    def get_language(self,text):
        credential = AzureKeyCredential(self.key)
        client = TextAnalyticsClient(endpoint=self.cognitive_service_url,credential=credential)

        detectedlanguage = client.detect_language(documents=[text])[0]
        print(detectedlanguage)
        print(detectedlanguage.primary_language.name)

    def get_text_translation(self, input_text):
        #cognitive_services_endpoint = os.environ['AZUREAI_COGNITIVESERVICES_URL']
        translator_endpoint = os.environ['AZUREAI_TRANSLATOR_URL']
        
        path = '/translate'
        constructed_url = translator_endpoint + path

        params['from'] = 'en'
        params['to']= ['fr', 'zu','zh-Hans']
        
        # You can pass more than one object in body.
        body = [{
            'text': input_text
        }]

        request = requests.post(constructed_url, params=params, headers=headers, json=body)
        response = request.json()


        print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))

    def chat_completions(self, input_text):
        endpoint = f"https://{self.based_endpoint}.openai.azure.com/"  
        deployment =  "gpt-35-turbo-16k"  
        subscription_key = self.key  
    
        # Initialize Azure OpenAI client with key-based authentication
        client = AzureOpenAI(  
            azure_endpoint=endpoint,  
            api_key=subscription_key,  
            api_version="2024-05-01-preview",  
        )  
      
        system_prompt = """
            You are an AI Historian that helps people find information anything about historical places,people or anything that 
            have importance in world history.

            Your answer should be accurate and if you dont know anything just say 'No I dont have any idea right now'

            Make your answer funny.
        """
        # Prepare the chat prompt  
        chat_prompt = [
            {
                "role": "system", "content": system_prompt,
                "role": "user", "content": input_text
            }
        ]  
    
        # Include speech result if speech is enabled  
        speech_result = chat_prompt  
        
        # Generate the completion  
        completion = client.chat.completions.create(  
            model=deployment,  
            messages=speech_result,  
            #past_messages=10,  
            max_tokens=800,  
            temperature=0.7,  
            top_p=0.95,  
            frequency_penalty=0,  
            presence_penalty=0,  
            stop=None,  
            stream=False  
        )  
        
        formatted_response = {
            "response": {
                "content": completion.choices[0].message.content,
                "role": completion.choices[0].message.role,
                "finish_reason": completion.choices[0].finish_reason
            },
            "metadata": {
                "model": completion.model,
                "timestamp": datetime.now().isoformat(),
                "prompt_tokens": completion.usage.prompt_tokens,
                "completion_tokens": completion.usage.completion_tokens,
                "total_tokens": completion.usage.total_tokens
            },
            "request_info": {
                "deployment": deployment,
                "max_tokens": 800,
                "temperature": 0.7,
                "top_p": 0.95
            }
        }

        #print(formatted_response)
        print(json.dumps(formatted_response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))
    
        

In [17]:
ai = AzureAIExamples(aiservice_type=AIServiceType.TEXT_TRANSLATION)

#### Text Translation

In [ ]:
ai.get_text_translation('What are the good tourist destinations in France')

#### Chat Completion

In [ ]:
result = ai.chat_completions('Where can you find the holy grail?')

#### Get Language

In [18]:
ai.get_language("Boun Jour")

{'id': '0', 'primary_language': DetectedLanguage(name=French, iso6391_name=fr, confidence_score=0.89), 'warnings': [], 'statistics': None, 'is_error': False, 'kind': 'LanguageDetection'}
French
